# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# store the CSV in WeatherPy into a DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kushmurun,52.4503,64.6311,66.38,66,10,8.10,KZ,1660163044
1,1,goma,-1.6792,29.2228,65.34,63,62,2.55,CD,1660163046
2,2,bluff,-46.6000,168.3333,41.68,69,98,8.10,NZ,1660163048
3,3,ponta do sol,32.6667,-17.1000,73.71,77,50,5.91,PT,1660162452
4,4,busselton,-33.6500,115.3333,51.62,80,40,4.56,AU,1660163053


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# configure our gmaps
gmaps.configure(api_key=g_key)

In [4]:
# heatmap of the humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()

# add the heatmap layer to the figure
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=300,
                                 point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
idealWeatherDF = city_data_df.loc[
    (city_data_df["Max Temp"] > 70) & (city_data_df["Max Temp"] < 80) & \
    (city_data_df["Wind Speed"] < 10) & \
    (city_data_df["Cloudiness"] == 0)
].dropna()

# print(len(idealWeatherDF))

idealWeatherDF.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,17,faanui,-16.4833,-151.7500,76.95,72,0,4.05,PF,1660163084
103,103,north bend,43.4065,-124.2243,77.23,56,0,9.22,US,1660163303
105,105,alexandria,31.2156,29.9553,79.34,70,0,7.05,EG,1660163307
118,118,deer lake,49.1667,-57.4316,71.91,40,0,4.50,CA,1660163337
299,299,madaba,31.5833,35.6667,73.56,67,0,4.50,JO,1660163788


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = idealWeatherDF[["City", "Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df

/var/folders/3n/d3xwtzgd3hxgcj1r1bbqhms40000gn/T/ipykernel_12488/3573417003.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Hotel Name
17,faanui,PF,-16.4833,-151.7500,
103,north bend,US,43.4065,-124.2243,
105,alexandria,EG,31.2156,29.9553,
118,deer lake,CA,49.1667,-57.4316,
299,madaba,JO,31.5833,35.6667,
302,gourdon,FR,44.7500,1.5000,
346,trapani,IT,37.8333,12.6667,
352,salisbury,GB,51.0693,-1.7957,
437,shirvan,AZ,39.9509,48.8994,
445,siirt,TR,37.9326,41.9403,


In [11]:
# Set the parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# loop through the dataframe using iterrows()
for index, row in hotel_df.iterrows():
    # get the latitude and longitude from the current row
    lat = row["Lat"]
    lng = row["Lng"]
    
    # use the latitude and longitude to set the location value in the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # use the google apis places base url to search
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print the url
    name_address = requests.get(base_url, params=params).json()
    
    #pprint(name_address) # print from ["results"][0]["name"]
    
    try:
        # populate the hotel name
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except:
        print(f"Error finding lodging in {row['City']}. Skipping...")
        pass

hotel_df

Error finding lodging in trapani. Skipping...


,City,Country,Lat,Lng,Hotel Name
17,faanui,PF,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
103,north bend,US,43.4065,-124.2243,Red Lion Hotel Coos Bay
105,alexandria,EG,31.2156,29.9553,Royal Crown Hotel
118,deer lake,CA,49.1667,-57.4316,"Holiday Inn Express Deer Lake, an IHG Hotel"
299,madaba,JO,31.5833,35.6667,مزرعة القيسي
302,gourdon,FR,44.7500,1.5000,Manoir la barriere
346,trapani,IT,37.8333,12.6667,
352,salisbury,GB,51.0693,-1.7957,Mercure Salisbury White Hart Hotel
437,shirvan,AZ,39.9509,48.8994,Nəriman Nərimanov küçəsi Şirvan Azərbaycan
445,siirt,TR,37.9326,41.9403,Barden Otel


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))